***
## Load package and define connection
***

In [1]:
import pyodbc
# Define the connection str
connstr = 'DRIVER=SQL Server;SERVER={};DATABASE={};Trusted_Connection=yes'
uid_login = [r'LAPTOP-K9RBHJ19\SQLEXPRESS','NTPU_CLASS']
connstr = connstr.format(*uid_login)
# create connection
conn = pyodbc.connect(connstr)
conn.autocommit = True
cursor = conn.cursor()

***
## Find all already exist DB
***

In [2]:
sqlstr = 'select name from master.sys.databases'
cursor.execute(sqlstr)
res = cursor.fetchall()
print(res)

[('master',), ('tempdb',), ('model',), ('msdb',), ('NTPU_CLASS',)]


***
## Delete a DB (Not in Use)
***

In [14]:
sqlstr = '''use master
            drop database NTPU_CLASS'''
cursor.execute(sqlstr)

***
## Create a DB
***

In [15]:
sqlstr = 'create database NTPU_CLASS'
cursor.execute(sqlstr)

***
## Find existance of a table
***

In [18]:
sqlstr = 'select name from NTPU_CLASS.sys.tables'
cursor.execute(sqlstr)
res = cursor.fetchall()
print(res)

[('Member_Info',)]


***
## Create a table
***

In [17]:
sqlstr = '''
create table [NTPU_CLASS].[dbo].[Member_Info] (
	MemberId char(6),
    RecordDate date,
    GameMode int,
    GameSession int,
    [Buy-in] int,
    OriginScore int,
    Discount int,
    FinalScore int,
    Mdate datetime,
    primary key (MemberId,RecordDate,GameMode,GameSession,
                 [Buy-in],OriginScore,Discount,FinalScore)
)
'''
cursor.execute(sqlstr)

***
## Insert data to table
***

In [1]:
import pandas as pd
import os
path = r'C:\Users\User\Desktop\北大統計-授課資料\113-2 日間部-Python程式設計\ReadList\capm5.csv'
df_sql = pd.read_csv(path)
df_sql.head()

,date,time,ge,ibm,ford,msft,dis,xom,mkt,riskfree
0,30JAN1998,1,0.056218,-0.056153,0.058842,0.154255,0.080884,-0.030644,0.004502,0.004480
1,27FEB1998,2,0.003226,0.059620,0.109069,0.136154,0.047368,0.081728,0.073303,0.004480
2,31MAR1998,3,0.112379,-0.005386,0.145856,0.056047,-0.046343,0.060784,0.051301,0.004519
3,30APR1998,4,-0.011603,0.115523,0.086181,0.006983,0.168337,0.080407,0.010903,0.004503
4,29MAY1998,5,-0.021277,0.015922,0.132333,-0.058946,-0.090818,-0.029461,-0.025748,0.004488


In [6]:
pd.to_datetime(df_sql['date'])

C:\Users\User\AppData\Local\Temp\ipykernel_12396\3578787445.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df_sql['date'])


0     1998-01-30
1     1998-02-27
2     1998-03-31
3     1998-04-30
4     1998-05-29
         ...    
175   2012-08-31
176   2012-09-28
177   2012-10-31
178   2012-11-30
179   2012-12-31
Name: date, Length: 180, dtype: datetime64[ns]

In [20]:
header = 'Insert into [NTPU_CLASS].[dbo].[Member_Info] values ('
for _,row in df_sql.iterrows():
    sqlstr = header + str(list(row))[1:-1] + ')'
    cursor.execute(sqlstr)

***
## Read from SQL
***

In [21]:
import pandas as pd
sqlstr = 'select * from [NTPU_CLASS].[dbo].[Member_Info]'
df = pd.read_sql(sqlstr,conn)
df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_10504\511252433.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sqlstr,conn)


,MemberId,RecordDate,GameMode,GameSession,Buy-in,OriginScore,Discount,FinalScore,Mdate
0,112434,2025-03-07,3400,2,1,4,0,4,2025-03-07 23:58:23
1,133992,2025-03-07,1200,1,2,4,0,4,2025-03-07 23:54:44
2,133992,2025-03-07,3400,2,1,4,-1,3,2025-03-07 23:58:23
3,331834,2025-03-07,1200,1,2,4,0,4,2025-03-07 23:54:44
4,331837,2025-03-07,3400,2,1,4,0,4,2025-03-07 23:58:23
